# Imports

In [5]:
import re
import numpy as np
import pandas as pd 

# Data Cleaning

In [155]:
# Loading data
data = pd.read_csv('product_hm.csv')

# product_id
data = data.dropna( subset=['product_id'], axis=0 )
data['product_id'] = data['product_id'].astype(int)

# product_category

# product_name
data['product_name'] = data['product_name'].apply( lambda x: x.replace(' ', '_').lower() if pd.notnull(x) else x )

# product_price 
data['product_price'] = data['product_price'].apply( lambda x: x.replace('$ ', '') if pd.notnull(x) else x )

# scrapy_datetime

# style_id
data['style_id'] = data['style_id'].astype(int)

# color_id
data['color_id'] = data['color_id'].astype(int)

# color_name
data['color_name'] = data['color_name'].apply( lambda x: x.replace('/', '_').replace(' ', '_').lower() if pd.notnull(x) else x )

# fit
data['fit'] = data['fit'].apply( lambda x: x.replace(' ', '_').lower() if pd.notnull(x) else x )

# size

## size_number
data['size_number'] =data['size'].apply( lambda x: re.search( r'\d+cm', x ).group(0) if pd.notnull(x) else x )
data['size_number'].apply( lambda x: re.search(r'\d+', x).group(0) if pd.notnull(x) else x )

## size_model
data['size_model'] = data['size'].str.extract( '(\d+/\\d+)' )

## composition
data = data.loc[~data['composition'].str.contains( 'Pocket lining:', na=False ), :]
data = data.loc[~data['composition'].str.contains( 'Shell:', na=False ), :]
data = data.loc[~data['composition'].str.contains( 'Pocket:', na=False ), :]



# drop duplicates
data = data.drop_duplicates( subset=['product_id', 'product_category', 'product_name', 'product_price',
                                     'scrapy_datetime', 'style_id', 'color_id', 'color_name', 'fit'], keep='last' )

# reset index 
data = data.reset_index(drop=True)

# reference
df_ref = pd.DataFrame( index=np.arange(len(data)), columns=['cotton', 'polyester', 'elastane', 'elasterell'] )

# break composition by comma
df1 = data['composition'].str.split(',', expand=True)

# ==== cotton ====
df_cotton = df1[0]
df_cotton.name = 'cotton'

df_ref = pd.concat([df_ref, df_cotton], axis=1)
df_ref = df_ref.loc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['cotton'] = df_ref['cotton'].fillna('Cotton 0%')

# Polyester
df_polyester = df1.loc[df1[1].str.contains('Polyester', na=False), 1]
df_polyester.name = 'polyester'

df_ref = pd.concat([df_ref, df_polyester], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['polyester'] = df_ref['polyester'].fillna( 'Polyester 0%' )

# elastane
df_elastane = df1.loc[df1[1].str.contains('Elastane', na=False), 1]
df_elastane.name = 'elastane'

## combine elastane from bath columns 1 and 2
df_elastane = df_elastane.combine_first( df1[2] )

df_ref = pd.concat([df_ref, df_elastane], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['elastane'] = df_ref['elastane'].fillna( 'Elastane 0%' )

# elasterell
df_elasterell = df1.loc[df1[1].str.contains('Elasterell', na=False), 1]
df_elasterell.name = 'elasterell'

df_ref = pd.concat([df_ref, df_elasterell], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['elasterell'] = df_ref['elasterell'].fillna( 'Elasterell 0%' )

# final join 
data = pd.concat( [data, df_ref], axis=1 )

# format format composition data
data['cotton'] = data['cotton'].apply( lambda x: int(re.search( '\d+', x ).group(0))/100 if pd.notnull(x) else x )
data['polyester'] = data['polyester'].apply( lambda x: int(re.search( '\d+', x ).group(0))/100 if pd.notnull(x) else x )
data['elastane'] = data['elastane'].apply( lambda x: int(re.search( '\d+', x ).group(0))/100 if pd.notnull(x) else x )
data['elasterell'] = data['elasterell'].apply( lambda x: int(re.search( '\d+', x ).group(0))/100 if pd.notnull(x) else x )


# drop columns
data = data.drop( ['size', 'product safety', 'composition'], axis=1 )

# drop duplicates
data = data.drop_duplicates()

In [157]:
data.head(10)

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,fit,size_number,size_model,cotton,polyester,elastane,elasterell
0,636207011,men_jeans_slim,slim_jeans,19.99,2021-09-06 10:59:36,636207,11,pale_denim_blue,slim_fit,187cm,31/30,0.88,0.1,0.02,0.0
1,636207011,men_jeans_slim,slim_jeans,19.99,2021-09-06 10:59:36,636207,11,dark_denim_blue,slim_fit,NaN,NaN,0.89,0.1,0.01,0.0
2,636207011,men_jeans_slim,slim_jeans,19.99,2021-09-06 10:59:36,636207,11,dark_gray_denim,slim_fit,NaN,NaN,0.89,0.1,0.01,0.0
3,636207011,men_jeans_slim,slim_jeans,19.99,2021-09-06 10:59:36,636207,11,gray,slim_fit,NaN,NaN,0.89,0.1,0.01,0.0
4,636207011,men_jeans_slim,slim_jeans,19.99,2021-09-06 10:59:36,636207,11,black,slim_fit,NaN,NaN,0.89,0.1,0.01,0.0
5,636207011,men_jeans_slim,slim_jeans,19.99,2021-09-06 10:59:36,636207,11,light_denim_blue,slim_fit,NaN,NaN,0.89,0.1,0.01,0.0
6,636207011,men_jeans_slim,slim_jeans,19.99,2021-09-06 10:59:36,636207,11,midnight_blue,slim_fit,NaN,NaN,0.89,0.1,0.01,0.0
7,636207011,men_jeans_slim,slim_jeans,19.99,2021-09-06 10:59:36,636207,11,dark_gray,slim_fit,NaN,NaN,0.89,0.1,0.01,0.0
8,636207011,men_jeans_slim,slim_jeans,19.99,2021-09-06 10:59:36,636207,11,denim_blue,slim_fit,NaN,NaN,0.89,0.1,0.01,0.0
9,636207011,men_jeans_slim,slim_jeans,19.99,2021-09-06 10:59:36,636207,11,white,slim_fit,NaN,NaN,0.89,0.1,0.01,0.0
